In [14]:
import vertica_python

connection_info = {
    'host': '127.0.0.1',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}

In [3]:
with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n5 -r5 cursor.execute("SELECT count(*) FROM views")

    for row in cursor.iterate():
        print(row)

33.7 ms ± 1.5 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
[10000000]


In [19]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n5 -r5 cursor.execute("SELECT count(DISTINCT user_id) FROM views")

3.2 s ± 91.5 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [20]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n5 -r5 cursor.execute("SELECT count(DISTINCT movie_id) FROM views")

3.16 s ± 104 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [21]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n5 -r5 cursor.execute("SELECT min(viewed_frame), max(viewed_frame) FROM views")

141 ms ± 31.3 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [40]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n5 -r5 cursor.execute("SELECT count(*), sum(viewed_frame), avg(viewed_frame) FROM views")

115 ms ± 3.15 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [23]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n1 -r5 cursor.execute("SELECT user_id, count(distinct movie_id) FROM views GROUP by user_id")

32.1 s ± 3.32 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [24]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n1 -r5 cursor.execute("SELECT movie_id, count(distinct user_id) FROM views GROUP by movie_id")

25.5 s ± 3.59 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [25]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n1 -r5 cursor.execute("SELECT user_id, sum(viewed_frame), min(viewed_frame), max(viewed_frame), avg(viewed_frame) FROM views GROUP by user_id")

25.9 s ± 4.51 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [26]:

with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    %timeit -n1 -r5 cursor.execute("SELECT movie_id, sum(viewed_frame), min(viewed_frame), max(viewed_frame), avg(viewed_frame) FROM views GROUP by movie_id")

17.1 s ± 4.91 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [5]:
from datetime import datetime, timedelta
import random
import string

def date_between(start, end):
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')

    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    random_microseconds = random.randint(1, int_delta)
    return start + timedelta(seconds=random_second, microseconds=random_microseconds)

def generate_movie_id(length=7):
    letters_and_digits = string.digits
    rand_string = ''.join(random.sample(letters_and_digits, length))
    return 'tt'+rand_string

def data_row(x=0):
    return [
            str(random.randint(1, 1000000)),
            generate_movie_id(),
            # возъмем среднюю продолжительность фильма 120 мин.
            # будем сохранять каждые 30 сек. просмотра
            random.randint(1, (60*60*2)/30),
            date_between('2022-05-23', '2022-05-30')
        ]

values = [r for r in map(data_row, range(0, 1000))]

In [7]:
with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()
    query = "INSERT INTO views (user_id, movie_id, viewed_frame, event_time) VALUES (%s,%s,%s,%s)"
    %timeit -n1 -r5 cursor.executemany(query, values)

57.9 ms ± 13.9 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [13]:
with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()
    %timeit -n1 -r1000 cursor.execute("INSERT INTO views (user_id, movie_id, viewed_frame, event_time) VALUES ('782951', 'tt5824071', 45, NOW())")

12.9 ms ± 1.03 ms per loop (mean ± std. dev. of 1000 runs, 1 loop each)


Тестирование обработки данных, поступающих в реальном времени

In [50]:
with vertica_python.connect(**connection_info) as connection:
    connection = vertica_python.connect(**connection_info)
    cursor = connection.cursor()

    for _ in range(3):
        %timeit -n5 -r5 cursor.execute("SELECT count(*), sum(viewed_frame), avg(viewed_frame) FROM views")
        for row in cursor.iterate():
            print(row)

230 ms ± 21.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
[10021218, 1207230376, 120.467429807435]
183 ms ± 16.1 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
[10021341, 1207245542, 120.467464583832]
209 ms ± 21.6 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
[10021479, 1207261373, 120.467385402893]
